In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import cudf
from category_encoders import TargetEncoder

from sklearn.model_selection import train_test_split
# from sklearn.model_selection import KFold

from cuml.ensemble import RandomForestRegressor 
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.preprocessing import OneHotEncoder

In [2]:
def cleandata(df):
    # Apply the categorization function to the transmission column
    df['Transmission_Category'] = df['transmission'].apply(categorize_transmission)
    #idea for future: include speed

    # Apply the parsing function to the engine column
    df[['HP', 'Volume', 'Cylinders', 'Cylinder_Type','Turbo']] = df['engine'].apply(parse_engine).apply(pd.Series)
    #idea: include
    
    
    # Apply the function to fill missing HP values
    df = fill_missing_hp(df)
    df = fill_missing_val(df, 'Volume')
    df = fill_missing_val(df, 'Cylinders')
    df = fill_missing_val(df, 'Cylinder_Type')


    df['HP'] = pd.to_numeric(df['HP'], errors='coerce')

    # Correct the fuel type for Tesla
    df['fuel_type'] = df.apply(correct_fuel_type, axis=1)
    
    cleanerdf = df.drop(columns=['id', 'engine', 'transmission'], inplace=False)
    df_cleaned = cleanerdf.dropna()

    # Apply the mapping to the columns
    df_cleaned.loc[:,'int_col'] = df_cleaned['int_col'].apply(group_colors)
    df_cleaned.loc[:,'ext_col'] = df_cleaned['ext_col'].apply(group_colors)
    if 'not supported' in df_cleaned['fuel_type']:
        df_cleaned.loc[df_cleaned['fuel_type']=='not supported', 'fuel_type'] = 'Gasoline'
    df_cleaned = df_cleaned[df_cleaned['fuel_type'] != '–']
    df_cleaned = df_cleaned[df_cleaned['int_col'] != '–']
    df_cleaned = df_cleaned[df_cleaned['ext_col'] != '–']
#     df_cleaned = df_cleaned[df_cleaned['price'] <= 2000000]
    df_cleaned = df_cleaned[df_cleaned['milage'] <= 200000]
    return df_cleaned


def group_colors(color):
    color = color.lower()
    color_groups = {
        'black': ['black', 'jet', 'charcoal'],
        'white': ['white', 'pearl', 'ivory'],
        'red': ['red', 'maroon', 'burgundy'],
        'blue': ['blue', 'navy', 'azure', 'cyan'],
        'green': ['green', 'olive', 'lime'],
        'yellow': ['yellow', 'gold', 'beige', 'sand'],
        'grey': ['grey', 'gray', 'silver', 'metallic'],
        'brown': ['brown', 'tan', 'chocolate'],
        'other': []
    }
    
    for main_color, sub_colors in color_groups.items():
        if any(sub_color in color for sub_color in sub_colors):
            return main_color
    return 'other'

def fill_missing_hp(df):
    df['HP'] = df.groupby(['model', 'brand', 'model_year'])['HP'].transform(
        lambda x: x.fillna(x.mean())
    )
    
    def get_closest_year_hp(row, df):
        model, brand, year = row['model'], row['brand'], row['model_year']
        mask = (df['model'] == model) & (df['brand'] == brand)
        available_years = df[mask & df['HP'].notna()]['model_year'].unique()
        
        if available_years.size == 0:
            return df['HP'].mode()[0]  # Use mode of all HPs if no match is found
        
        closest_year = available_years[np.abs(available_years - year).argmin()]
        closest_year_hp = df[mask & (df['model_year'] == closest_year)]['HP'].mean()
        
        return closest_year_hp
    
    df['HP'] = df.apply(
        lambda row: get_closest_year_hp(row, df) if pd.isnull(row['HP']) else row['HP'], axis=1
    )
    
    return df

def fill_missing_val(df, col):
    
    df[col] = df.groupby(['model', 'brand', 'model_year'])[col].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
    )
    
    def get_closest_year_val(row, df, col):
        model, brand, year = row['model'], row['brand'], row['model_year']
        mask = (df['model'] == model) & (df['brand'] == brand)
        available_years = df[mask & df[col].notna()]['model_year'].unique()
        
        if available_years.size == 0:
            return df[col].mode()[0]  # Use mode of all HPs if no match is found
        
        closest_year = available_years[np.abs(available_years - year).argmin()]
        closest_year_val = df[mask & (df['model_year'] == closest_year)][col].mode()
        if closest_year_val.empty:
            return df[col].mode()[0]
        
        return closest_year_val[0]
    
    df[col] = df.apply(
        lambda row: get_closest_year_val(row, df, col) if pd.isnull(row[col]) else row[col], axis=1
    )
    
    return df

def preprocess_categorical(df, columns):
    for col in columns:
        df[col] = df[col].fillna('Missing').astype(str)
    return df

def group_rare_categories(series, threshold):
    counts = series.value_counts()
    rare = counts[counts < threshold].index
    return series.apply(lambda x: 'Other' if x in rare else x)

# Function to parse engine details
def parse_engine(engine_str):
    # Regular expressions for extracting the details
    hp_pattern = r'(\d+\.?\d*)HP'
    volume_pattern = r'(\d+\.?\d*)L'
    cylinders_acronym_pattern = r'(V|I|H)(\d+)'
    type_pattern = r'(V|Straight|I|H)?\s*(\d+)\s*Cylinders?'

    # Extracting the horsepower
    hp = re.search(hp_pattern, engine_str)
    hp = float(hp.group(1)) if hp else None
    
    if 'electric motor' in engine_str.lower() or 'electric fuel' in engine_str.lower():
        return hp, 'Electric', 'Electric', 'Electric', 0
    
    
    # Extracting the volume
    volume = re.search(volume_pattern, engine_str)
    volume = str(volume.group(1)) if volume else None
    

    # Extracting the number and type of cylinders
    cylinders_match = re.search(cylinders_acronym_pattern, engine_str)
    if cylinders_match:
        cylinder_type = 'V' if cylinders_match.group(1) == 'V' else 'Inline'
        cylinders = int(cylinders_match.group(2))
        if cylinders_match.group(1) == 'H':
                cylinder_type = 'Straight'
    else:
        cylinders_match = re.search(type_pattern, engine_str)
        if cylinders_match:
            cylinder_type = cylinders_match.group(1) if cylinders_match.group(1) else ''
            cylinders = str(cylinders_match.group(2))
            cylinder_type = cylinder_type or ('V' if 'V' in engine_str else 'Straight' if 'Straight' in engine_str else 'Inline')
        else:
            cylinders = None
            cylinder_type = None

    turbo = 'Y' if 'Turbo' in engine_str or 'turbo' in engine_str else 'N'

    return hp, volume, cylinders, cylinder_type, turbo

# Function to correct fuel type for Tesla
def correct_fuel_type(row):
    if 'Tesla' in row['brand'] or 'electric motor' in row['engine'].lower():
        return 'Electric'
    return row['fuel_type']

# Function to categorize transmission
def categorize_transmission(transmission_str):
    transmission_str = transmission_str.lower()
    if 'manual' in transmission_str or 'm/t' in transmission_str:
        return 'Manual'
    elif 'dual' in transmission_str or 'dual shift' in transmission_str:
        return 'Dual Shift'
    elif 'cvt' in transmission_str:
        return 'CVT'
    else:
        return 'Auto'

In [3]:
df = pd.read_csv('/kaggle/input/kagglex-skill-assessment-challenge/train.csv')
df.describe()
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [76]:
# Calculate the percentage of missing values in each column
# missing_percentage = df.isnull().mean() * 100

# # Display the percentage of missing values
# print(missing_percentage)

id                       0.000000
brand                    0.000000
model                    0.000000
model_year               0.000000
milage                   0.000000
fuel_type                0.000000
engine                   0.000000
transmission             0.000000
ext_col                  0.000000
int_col                  0.000000
accident                 0.000000
clean_title              0.000000
price                    0.000000
Transmission_Category    0.000000
HP                       7.475172
Volume                   0.794133
Cylinders                0.884418
Cylinder_Type            0.884418
Turbo                    0.000000
dtype: float64


In [4]:
categorical_cols = ['brand', 'model', 'fuel_type', 'ext_col', 'int_col', 'accident', 'clean_title', 'Volume','Cylinders','Cylinder_Type', 'Transmission_Category', 'Turbo']

cleaner_df = cleandata(df)
df_cleaned = cleaner_df[cleaner_df['price']<= 2000000]
df_cleaned.loc[:,'model'] = group_rare_categories(df_cleaned['model'], threshold=2)
df_cleaned.loc[:,'brand'] = group_rare_categories(df_cleaned['brand'], threshold=10)

/tmp/ipykernel_34/4168761369.py:85: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
/tmp/ipykernel_34/4168761369.py:85: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
/tmp/ipykernel_34/4168761369.py:85: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
/tmp/ipykernel_34/4168761369.py:85: UserWarning: Unable to sort modes:

In [5]:
# Calculate the percentage of missing values in each column
missing_percentage = df_cleaned.isnull().mean() * 100

# Display the percentage of missing values
print(missing_percentage)

brand                    0.0
model                    0.0
model_year               0.0
milage                   0.0
fuel_type                0.0
ext_col                  0.0
int_col                  0.0
accident                 0.0
clean_title              0.0
price                    0.0
Transmission_Category    0.0
HP                       0.0
Volume                   0.0
Cylinders                0.0
Cylinder_Type            0.0
Turbo                    0.0
dtype: float64


In [6]:
# Split the data into training and testing sets
X = df_cleaned.drop('price', axis=1)
X = X.drop(columns=['int_col'])##########################

y = df_cleaned['price']

encoder = TargetEncoder()
X_encoded = encoder.fit_transform(X, y)

X_train_full, X_test, y_train_full, y_test = train_test_split(X_encoded, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2222, random_state=42)  # 0.2222 * 0.9 = 0.2

# Convert to cuDF
X_train_cudf = cudf.DataFrame.from_pandas(X_train)
X_val_cudf = cudf.DataFrame.from_pandas(X_val)
X_test_cudf = cudf.DataFrame.from_pandas(X_test)
y_train_cudf = cudf.Series(y_train.values)
y_val_cudf = cudf.Series(y_val.values)
y_test_cudf = cudf.Series(y_test.values)


# Train the Random Forest model
final_model = RandomForestRegressor(n_estimators=30, max_depth=5, random_state=42)
final_model.fit(X_train_cudf, y_train_cudf)

# Evaluate on the validation set
val_predictions = final_model.predict(X_val_cudf)
val_score = final_model.score(X_val_cudf, y_val_cudf)
print(f"Validation Score: {val_score}")

/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Validation Score: 0.2035170472719463


In [148]:
# Finding optimal num trees and depth------------------------------------------------------

from sklearn.model_selection import GridSearchCV
from cuml.ensemble import RandomForestRegressor as cuRF
import cudf
import numpy as np

class cuMLWrapper:
    def __init__(self, **params):
        self.model = cuRF(**params)

    def fit(self, X, y):
        self.model.fit(cudf.DataFrame.from_pandas(X), cudf.Series(y))
        return self

    def predict(self, X):
        return self.model.predict(cudf.DataFrame.from_pandas(X)).get()

    def score(self, X, y):
        return self.model.score(cudf.DataFrame.from_pandas(X), cudf.Series(y))

    def get_params(self, deep=True):
        return self.model.get_params(deep)

    def set_params(self, **params):
        self.model.set_params(**params)
        return self

# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [10, 50, 100, 150, 200],  # Example values, adjust as needed
    'max_depth': [5, 10, 15, 20]  # Example values, adjust as needed
}

# Convert data to cuDF
X_train_full_cudf = cudf.DataFrame.from_pandas(X_train_full)
y_train_full_cudf = cudf.Series(y_train_full.values)

# Initialize the model
rf_model = cuMLWrapper()

# Perform Grid Search
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_full, y_train_full)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Train the final model with the best parameters
best_model = cuRF(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'])
best_model.fit(X_train_cudf, y_train_cudf)

# Evaluate on the validation set with the best model
val_predictions_best = best_model.predict(X_val_cudf)
val_score_best = best_model.score(X_val_cudf, y_val_cudf)
print(f"Validation Score with Best Model: {val_score_best}")

/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/cuml/internals/

Best parameters found: {'max_depth': 5, 'n_estimators': 50}
Validation Score with Best Model: 0.2031629092517635


In [149]:
# Load the test data
test_file_path = '/kaggle/input/kagglex-skill-assessment-challenge/test.csv'  # Replace with your actual file path
test_df = pd.read_csv(test_file_path)
test_ids = test_df['id']

 # Apply the categorization function to the transmission column
test_df['Transmission_Category'] = test_df['transmission'].apply(categorize_transmission)
#idea for future: include speed

# Apply the parsing function to the engine column
test_df[['HP', 'Volume', 'Cylinders', 'Cylinder_Type','Turbo']] = test_df['engine'].apply(parse_engine).apply(pd.Series)
#idea: include

test_df = fill_missing_hp(test_df)
test_df = fill_missing_val(test_df, 'Volume')
test_df = fill_missing_val(test_df, 'Cylinders')
test_df = fill_missing_val(test_df, 'Cylinder_Type')

test_df['HP'] = pd.to_numeric(test_df['HP'], errors='coerce')

# Correct the fuel type for Tesla
test_df['fuel_type'] = test_df.apply(correct_fuel_type, axis=1)

if 'not supported' in test_df['fuel_type']:
    test_df.loc[test_df['fuel_type']=='not supported', 'fuel_type'] = 'Gasoline'
    
# Apply the mapping to the columns
test_df.loc[:,'int_col'] = test_df['int_col'].apply(group_colors)
test_df.loc[:,'ext_col'] = test_df['ext_col'].apply(group_colors)

df_cleaned_test = test_df.drop(columns=['id', 'engine', 'transmission' , 'int_col'], inplace=False)

df_cleaned_test.loc[:,'model'] = group_rare_categories(df_cleaned_test['model'], threshold=2)
df_cleaned_test.loc[:,'brand'] = group_rare_categories(df_cleaned_test['brand'], threshold=10)

/tmp/ipykernel_34/2297922468.py:87: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
/tmp/ipykernel_34/2297922468.py:87: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
/tmp/ipykernel_34/2297922468.py:87: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
/tmp/ipykernel_34/2297922468.py:87: UserWarning: Unable to sort modes:

In [142]:
df_cleaned['int_col'].value_counts()

int_col
black     32064
yellow     7864
grey       6045
brown      2271
other      1873
red        1536
white      1188
blue        226
green        53
Name: count, dtype: int64

In [70]:
test_df[test_df.isnull().any(axis=1)]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,Transmission_Category,HP,Volume,Cylinders,Cylinder_Type,Turbo
16,54289,Toyota,Tundra SR5,2023,2925,Gasoline,3.4L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,grey,black,None reported,Yes,Auto,NaN,3.4,6,V,Y
89,54362,Toyota,Land Cruiser Base,2023,7900,–,–,10-Speed A/T,green,white,None reported,Yes,Auto,NaN,None,None,None,N
108,54381,Rolls-Royce,Cullinan,2022,7928,Gasoline,6.7L V12 48V GDI DOHC Twin Turbo,8-Speed Automatic,other,white,None reported,Yes,Auto,NaN,6.7,12,V,Y
115,54388,Ford,F-150 XL,2023,2823,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,Automatic,black,black,None reported,Yes,Auto,NaN,3.5,6,V,Y
161,54434,Mazda,CX-9 Carbon Edition,2021,36844,Gasoline,2.5L I4 16V GDI DOHC Turbo,8-Speed Automatic,grey,yellow,None reported,Yes,Auto,NaN,2.5,4,Inline,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35960,90233,Mazda,Mazda3 Grand Touring,2019,70891,Gasoline,2.5L I4 16V GDI DOHC,6-Speed Automatic,white,other,None reported,Yes,Auto,NaN,2.5,4,Inline,N
36010,90283,Ford,F-150 XLT SuperCab,1993,195000,–,–,A/T,blue,grey,None reported,Yes,Auto,NaN,None,None,None,N
36081,90354,Bentley,Bentayga Azure First Edition,2023,1335,Gasoline,4.0L V8 32V GDI DOHC Twin Turbo,8-Speed Automatic,black,other,None reported,Yes,Auto,NaN,4.0,8,V,Y
36115,90388,Chevrolet,Colorado Z71,2019,71536,Gasoline,3.6L V6 24V GDI DOHC,Automatic,black,black,None reported,Yes,Auto,NaN,3.6,6,V,N


In [95]:
# Calculate the percentage of missing values in each column
# missing_percentage = df_cleaned.isnull().mean() * 100

# # Display the percentage of missing values
# print(missing_percentage)

brand                    0.0
model                    0.0
model_year               0.0
milage                   0.0
fuel_type                0.0
ext_col                  0.0
int_col                  0.0
accident                 0.0
clean_title              0.0
Transmission_Category    0.0
HP                       0.0
Volume                   0.0
Cylinders                0.0
Cylinder_Type            0.0
Turbo                    0.0
dtype: float64


In [164]:
# Encode categorical variables in the test set using the already fitted encoder
X_test_encoded = encoder.transform(df_cleaned_test)

# Convert the encoded test set to cuDF
X_test_encoded_cudf = cudf.DataFrame.from_pandas(X_test_encoded)

# Predict on the test set using the final model
test_predictions = final_model.predict(X_test_encoded_cudf)

# print(test_predictions)
# Create a DataFrame with original ID and prediction
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'price': test_predictions.to_numpy() # Convert cuDF Series to numpy array
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('random_forest_submission_nointcol30.csv', index=False)

print(submission_df.head())


      id         price
0  54273  22182.981172
1  54274  21856.438365
2  54275  24834.897483
3  54276  53706.708440
4  54277  43369.470904


In [146]:
import pandas as pd
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming X_encoded is the DataFrame with your features after encoding
# Add a constant term for the intercept in the regression model
X_with_const = sm.add_constant(X_encoded)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['feature'] = X_with_const.columns
vif_data['VIF'] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]

# Display the VIF data
print(vif_data)

                  feature            VIF
0                   brand       1.474554
1                   model       2.394886
2              model_year       1.939063
3                  milage       1.940299
4               fuel_type       1.085322
5                 ext_col       1.024866
6                 int_col       1.127283
7                accident       1.113962
8             clean_title  271073.077235
9   Transmission_Category       1.064012
10                     HP       2.841948
11                 Volume       1.928271
12              Cylinders       1.836618
13          Cylinder_Type       1.090324
14                  Turbo       1.092463
